# Autoimplant

Perform autoimplant from a segmented skull file in a Nifti format.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/biodatlab/autoimplantpipe/blob/master/notebooks/02_autoimplant_prediction.ipynb)

In [ ]:
import os
import os.path as op
import nibabel as nib
from pathlib import Path
from itkwidgets import view
from autoimplantpipe import (
    load_autoimplant_model,
    predict_autoimplant,
    save_prediction_array_to_nifti,
    convert_nifti_to_stl,
)

In [ ]:
# Define model
model_path = "3DUNetCNN_SkullBreak+in-house.pth"
autoimplant_model = load_autoimplant_model(model_path)
autoimplant_model.eval()

In [ ]:
# Define .nii file to predict and output folder
defective_path = "skullbreak_parietotemporal_001.nii"
parent_path = "out_autoimplant"

if not op.exists(parent_path):
    os.makedirs(parent_path, exist_ok=True)

output_path = op.join(
    parent_path, op.splitext(Path(defective_path).stem)[0] + "_autoimplant.nii"
)
stl_skull_path = op.join(
    parent_path, op.splitext(Path(defective_path).stem)[0] + "_autoimplant_skull.stl"
)
stl_implant_path = op.join(
    parent_path, op.splitext(Path(defective_path).stem)[0] + "_autoimplant_implant.stl"
)

In [ ]:
test_file = {"image": defective_path, "label": defective_path}
prediction_output = predict_autoimplant(test_file, autoimplant_model)

In [ ]:
save_prediction_array_to_nifti(prediction_output, defective_path, output_path)
convert_nifti_to_stl(output_path, stl_skull_path, label=1)
convert_nifti_to_stl(output_path, stl_implant_path, label=2)

### Preview Prediction

In [ ]:
img_output = nib.load(output_path)  # read prediction .nii file
img_output = img_output.get_fdata()
view(img_output)